# 1.0 Imports

In [3]:
import pandas as pd
import numpy as np
import os

# 2.0 Read CSV
**Nesta etapa, o arquivo csv é lido e atribuido à classe pandas. Também utilizo a função head() para me ajudar a enxergar como os dados estão dispostos, com isso posso gerar ideias de como fazer a próxima etapa.**

In [4]:
dir_path = os.path.dirname(os.path.abspath(''))
csv_path = os.path.join(dir_path,'dataset','teste_indicium_precificacao.csv')
data_frame = pd.read_csv(csv_path)
data_frame.head()

,id,nome,host_id,host_name,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,ultima_review,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
1,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
2,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
3,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
4,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129


# 3.0 Data Processing
**Após um vislumbre inicial dos dados, é importante aprofundar o exame para obter informações mais detalhadas sobre o DataFrame.**

**Ao examinar a quantidade de linhas, identificar valores ausentes e verificar os tipos de dados em cada coluna, consigo extrair insights valiosos que orientam as próximas etapas de limpeza e preparação dos dados.**

**É evidente que as colunas do dataframe não estão padronizadas, algumas com nome em inglês outras em português. Também é interessante organizar para que tudo fique padronizado, como a base de dados é de estabelecimentos de Nova York vou deixá-los todos em Inglês.**

In [31]:
# print para saber as informações do data frame
print(data_frame.info())

# padronização dos nomes das colunas
# criação de um dict com a troca de nomes das colunas
colunas_renomeadas = {'nome':'name',
                      'bairro_group':'district',
                      'bairro':'neighborhood',
                      'minimo_noites':'minimum_nights',
                      'numero_de_reviews':'reviews_count',
                      'ultima_review':'last_review',
                      'reviews_por_mes':'monthly_reviews',
                      'calculado_host_listings_count':'calculated_host_listings_count',
                      'disponibilidade_365':'yearly_availability'}

data_frame.rename(columns=colunas_renomeadas, inplace=True)

# eliminação de valores vazios das colunas identificadas
for coluna in ['name', 'host_name', 'last_review', 'monthly_reviews']:
    data_frame.dropna(subset=[coluna], inplace=True)

# transformar coluna ultima_review em date time
data_frame['last_review'] = pd.to_datetime(data_frame['last_review'], format='%Y-%m-%d')

# criação de uma coluna chamada preço total que é o price multiplicado pela quantidade mínima de noites
data_frame['total_price'] = data_frame['price']*data_frame['minimum_nights']




# info do data frame após as eliminações das linhas null
print(data_frame.info())



<class 'pandas.core.frame.DataFrame'>
Index: 38820 entries, 0 to 48851
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              38820 non-null  int64         
 1   name                            38820 non-null  object        
 2   host_id                         38820 non-null  int64         
 3   host_name                       38820 non-null  object        
 4   district                        38820 non-null  object        
 5   neighborhood                    38820 non-null  object        
 6   latitude                        38820 non-null  float64       
 7   longitude                       38820 non-null  float64       
 8   room_type                       38820 non-null  object        
 9   price                           38820 non-null  int64         
 10  minimum_nights                  38820 non-null  int64         
 11  reviews

# Analise

In [24]:
maximo_numero_reviews = data_frame['numero_de_reviews'].max()
minimo_numero_reviews = data_frame['numero_de_reviews'].min()
media_numero_reviews = data_frame['numero_de_reviews'].mean()
mediana = data_frame['numero_de_reviews'].median()

media_disponibilidade = data_frame['disponibilidade_365'].mean()
print(maximo_numero_reviews,'\n',minimo_numero_reviews,'\n',media_numero_reviews,'\n',mediana)

df_disponibilidade_abaixo_media = data_frame[data_frame['disponibilidade_365'] < media_disponibilidade]

df_lugares_bons_de_alugar = df_disponibilidade_abaixo_media[df_disponibilidade_abaixo_media['numero_de_reviews'] > media_numero_reviews]
df_lugares_bons_de_alugar = df_lugares_bons_de_alugar[['nome','price','minimo_noites','numero_de_reviews','disponibilidade_365']]
df_lugares_bons_de_alugar

629 
 1 
 29.290777949510563 
 9.0


,nome,price,minimo_noites,numero_de_reviews,disponibilidade_365
5,BlissArtsSpace!,60,45,49,0
7,Cozy Clean Guest Room - Family Apt,79,2,118,0
9,Beautiful 1br on Upper West Side,135,5,53,6
10,Central Manhattan/near Broadway,85,2,188,39
14,Only 2 stops to Manhattan studio,140,2,148,46
...,...,...,...,...,...
42664,Queens Studio.,55,2,37,6
42941,Lucio's Room #3 a unos minutos del JFK y LGA,75,1,31,89
43138,Comfortable Room with Private Bath - 15 min to...,50,1,34,61
43790,NYC High End Upper East Side Central Park Gem ...,151,1,30,42


In [32]:
data_frame[data_frame['host_id'] == 2845]

,id,name,host_id,host_name,district,neighborhood,latitude,longitude,room_type,price,minimum_nights,reviews_count,last_review,monthly_reviews,calculated_host_listings_count,yearly_availability,preco_total,total_price
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,225,225
22727,18393354,Midtown Sanctuary,2845,Jennifer,Manhattan,Midtown,40.75358,-73.99190,Shared room,99,1,1,2018-07-18,0.08,2,365,99,99
